In [1]:
import cohere

# Get your cohere API key on: www.cohere.com
co = cohere.Client("HZFWyqSeeJXruZksq0uYjM3gqS4zcGbXMqeL7NPG")

# Example query and passages
query = "What is the capital of the United States?"
documents = [
   "Carson City is the capital city of the American state of Nevada. At the  2010 United States Census, Carson City had a population of 55,274.",
   "The Commonwealth of the Northern Mariana Islands is a group of islands in the Pacific Ocean that are a political division controlled by the United States. Its capital is Saipan.",
   "Charlotte Amalie is the capital and largest city of the United States Virgin Islands. It has about 20,000 people. The city is on the island of Saint Thomas.",
   "Washington, D.C. (also known as simply Washington or D.C., and officially as the District of Columbia) is the capital of the United States. It is a federal district. ",
   "Capital punishment (the death penalty) has existed in the United States since before the United States was a country. As of 2017, capital punishment is legal in 30 of the 50 states.",
   "North Dakota is a state in the United States. 672,591 people lived in North Dakota in the year 2010. The capital and seat of government is Bismarck."
   ]

results = co.rerank(query=query, documents=documents, top_n=3, model="rerank-multilingual-v2.0")

In [2]:
results

[RerankResult<document['text']: Washington, D.C. (also known as simply Washington or D.C., and officially as the District of Columbia) is the capital of the United States. It is a federal district. , index: 3, relevance_score: 0.9999989>, RerankResult<document['text']: Charlotte Amalie is the capital and largest city of the United States Virgin Islands. It has about 20,000 people. The city is on the island of Saint Thomas., index: 2, relevance_score: 0.9003547>, RerankResult<document['text']: Carson City is the capital city of the American state of Nevada. At the  2010 United States Census, Carson City had a population of 55,274., index: 0, relevance_score: 0.6807725>]

In [5]:
results[0]

RerankResult<document['text']: Washington, D.C. (also known as simply Washington or D.C., and officially as the District of Columbia) is the capital of the United States. It is a federal district. , index: 3, relevance_score: 0.9999989>

In [19]:
hit = results[0]
print(hit.document["text"])
print(hit.index)
print(hit.relevance_score)

Washington, D.C. (also known as simply Washington or D.C., and officially as the District of Columbia) is the capital of the United States. It is a federal district. 
3
0.9999989


In [20]:
sections = ["J.(E)", "J.(G)","B.16(A)", "B.9", "B.4(A)", "I.3(B)(ii)","B.2(C)(i)", "B.1(D)(vi)", "B.1(F)", "J.(G)(iii)", "B.1(B)"]

top_scores = [0.0881, 0.0894, 0.1354, 0.1405, 0.1443, 0.1450, 0.1452, 0.1473, 0.1474, 0.1483, 0.1491]

In [6]:
from src.chat_bot import ExconManual
ad = True
#ad = False # = ADLA chat

logging_level = 15
#logging_level = 20
log_file = ''

if ad:
    path_to_manual_as_csv_file = "./inputs/ad_manual.csv"
    path_to_definitions_as_parquet_file = "./inputs/ad_definitions.parquet"
    path_to_index_as_parquet_file = "./inputs/ad_index.parquet"
    chat_for_ad = True
else:
    path_to_manual_as_csv_file = "./inputs/adla_manual.csv"
    path_to_definitions_as_parquet_file = "./inputs/adla_definitions.parquet"
    path_to_index_as_parquet_file = "./inputs/adla_index.parquet"
    chat_for_ad = False
    #excon = ExconManual(path_to_manual_as_csv_file, path_to_definitions_as_parquet_file, path_to_index_as_parquet_file, chat_for_ad = False,log_file=log_file, logging_level=logging_level)


excon = ExconManual(path_to_manual_as_csv_file, path_to_definitions_as_parquet_file, path_to_index_as_parquet_file, chat_for_ad = chat_for_ad, log_file=log_file, logging_level=logging_level)



In [ ]:
#user_context = "A non-resident wants to buy residential property in South Africa. What are the restrictions around doing this?"
#user_context = "A South African citizen is emigrating. What is the process they need to follow to take all their cash offshore?"
#user_context = "What is the Foreign investment limits for South African institutional investors?"
#user_context = "Can a individual transfer his shares in a dual listed company to his offshore portfolio?"
#user_context = "Can cash balances for individuals who are no longer tax resident be transferred offshore?"
#user_context = "How do I need to report outbound cross-boarder payments?"
#user_context = "Are there different categories for ALDAs?"
#user_context = "Who can trade gold?"

#user_context = "What documentary evidence is required?"
#user_context = "What does cemad say about travel services?"
#user_context = "Can I import gold?"
#user_context = "how much money can an individual take offshore?"
#user_context = "How much can an individual take offshore in a year?"
#user_context = "what is the limit for commodities"
#user_context = "How much can adult residents use for travel?"


query = "How do I need to report outbound cross-boarder payments?"
query = "What are prudential limits?"
query = "what does the regulation state about travel?"



In [95]:
from src.embeddings import get_ada_embedding, get_closest_nodes, num_tokens_from_string

query = "How much can an individual take offshore in a year?"

question_embedding = get_ada_embedding(query)        
threshold = 0.15
relevant_definitions = get_closest_nodes(excon.df_definitions_all, embedding_column_name = "Embedding", question_embedding = question_embedding, threshold = threshold)
relevant_sections = get_closest_nodes(excon.df_text_all, embedding_column_name = "Embedding", question_embedding = question_embedding, threshold = threshold)

print(f"Found {len(relevant_definitions)} definitions below the threshold")
for index, row in relevant_definitions.iterrows():
    print(f'{row["cosine_distance"]:.4f}: ({row["source"]:>10}): {row["Definition"]}')

documents = []
print(f'Found {len(relevant_sections)} sections below the threshold')
for index, row in relevant_sections.iterrows():
    print(f'{row["cosine_distance"]:.4f}: {row["section"]:>20}: {row["source"]:>15}: {row["text"]}')

unique_sections = relevant_sections.drop_duplicates(subset = 'section').copy()
unique_sections['document'] = unique_sections.section.apply(excon.get_regulation_detail)
#df_definitions, df_search_sections = excon.similarity_search(query, threshold)



Found 0 definitions below the threshold
Found 9 sections below the threshold
0.0266:            B.4(G)(i):          manual: How much money can an individual take offshore?
0.1341:            B.4(G)(i):        question: What are the yearly spending limits for residents temporarily abroad?
0.1412:             H.(H)(v):        question: Are there restrictions on the amount that exchange traded funds can invest offshore?
0.1427:            B.2(B)(i):        question: What is the maximum amount I can transfer for foreign investments in a calendar year?
0.1434:              B.17(E):        question: Can I keep foreign currency offshore?
0.1448:           B.3(B)(iv):        question: Can income from trusts be transfered offshore?
0.1458:          B.5(B)(iii):        question: Can an immigrants to acquire foreign currency to pay offshore obligations?
0.1464:              B.17(B):        question: What do I do if I inherited offshore assets?
0.1482:           B.2(C)(ii):        question: Can pr

In [90]:
results = co.rerank(query=query, documents=unique_sections["document"].to_list(), top_n=11, model="rerank-multilingual-v2.0")

for i in range(len(unique_sections)):
    original_index = results[i].index
    print(f"Section {unique_sections.iloc[original_index]['section']:>20} had score {unique_sections.iloc[original_index]['cosine_distance']:.4f}, now is ranked {results[i].relevance_score:.4f}")


Section            B.2(B)(i) had score 0.1427, now is ranked 0.0049
Section            B.4(G)(i) had score 0.0266, now is ranked 0.0035
Section           B.2(C)(ii) had score 0.1482, now is ranked 0.0004
Section              B.17(E) had score 0.1434, now is ranked 0.0000
Section           B.3(B)(iv) had score 0.1448, now is ranked 0.0000
Section              B.17(B) had score 0.1464, now is ranked 0.0000
Section          B.5(B)(iii) had score 0.1458, now is ranked 0.0000
Section             H.(H)(v) had score 0.1412, now is ranked 0.0000


In [97]:
import pandas as pd

summary_data = excon.df_text_all[excon.df_text_all["source"] == 'summary']
new_rows = []
for index, row in unique_sections.iterrows():
    section = row['section']
    summary = summary_data[summary_data["section"] == section]
    if len(summary) > 1:
        for n in range(len(summary)):
            new_row = row.copy()
            new_row['document'] = summary.iloc[n]['text']
            new_rows.append(new_row)
    else:
        new_rows.append(row)

unique_sections = pd.DataFrame(new_rows)
#unique_sections
for index, row in unique_sections.iterrows():
    section = row['section']
    summary = summary_data[summary_data["section"] == section]
    if len(summary) != 1:
        print(f'Section {section} has {len(summary)} parts')
    else:        
        unique_sections.at[index, 'document'] = summary.iloc[0]['text']

results = co.rerank(query=query, documents=unique_sections["document"].to_list(), top_n=11, model="rerank-multilingual-v2.0")

for i in range(len(unique_sections)):
    original_index = results[i].index
    print(f"Section {unique_sections.iloc[original_index]['section']:>20} had score {unique_sections.iloc[original_index]['cosine_distance']:.4f}, now is ranked {results[i].relevance_score:.4f}")

In [110]:
print(excon.get_regulation_detail('B.4(G)(i)'))

B.4 Single discretionary allowance and other miscellaneous payments for private individuals
    (G) Residents temporarily abroad
        (i) Subsistence allowance
            (a) Residents temporarily abroad, may avail of the R1 million single discretionary allowance and the R10 million foreign capital allowance per calendar year without returning to South Africa.
            (b) The requirements under section B.2(B)(i) of the Authorised Dealer Manual and subsection (A) above must be adhered to.
            (c) Where residents temporarily abroad make use of a general or special power of attorney to facilitate such transfers, a certified copy of the applicant's valid green barcoded identity document or Smart identity document card must accompany the power of attorney.
            (d) In terms of the TCS system, a TCS PIN letter will be issued to the taxpayer that will contain the tax number and TCS PIN. Authorised Dealers must use the TCS PIN to verify the taxpayer's tax compliance stat

In [78]:
summary_data[summary_data["section"].str.startswith('B.4(B)(i)')]

,section,text,Embedding,source,cosine_distance
539,B.4(B)(i),There is no limit on the amount of Rand that c...,"[0.026708412915468216, -0.01266176626086235, 0...",summary,0.191584


In [66]:
for i in range(len(unique_sections)):
    original_index = results[i].index
    print(f"Section {unique_sections.iloc[original_index]['section']:>20} had score {unique_sections.iloc[original_index]['cosine_distance']:.4f}, now is ranked {results[i].relevance_score:.4f}")

Section            B.2(I)(i) had score 0.0798, now is ranked 0.7856
Section           B.2(I)(iv) had score 0.1435, now is ranked 0.3523
Section            B.2(I)(v) had score 0.1256, now is ranked 0.0213
Section           B.2(H)(iv) had score 0.1364, now is ranked 0.0049


In [36]:
section = excon.get_regulation_detail('B.4(A)')
print(f'Number of tokens: {num_tokens_from_string(section)}')
print(section)


Number of tokens: 710
B.4 Single discretionary allowance and other miscellaneous payments for private individuals
    (A) Single discretionary allowance per calendar year
        (i) Residents (natural persons) who are 18 years and older may be permitted to avail of a single discretionary allowance within an overall limit of R1 million per individual per calendar year without the requirement to obtain a TCS PIN letter, which may be used for any legal purpose abroad (including for investment purposes as well as the sending of gift parcels in lieu of cash excluding gold and jewellery). Authorised Dealers must ensure that the purpose for which the funds are used abroad must be reported under the correct category on the FinSurv Reporting System.
        (ii) This dispensation may be utilised solely at the discretion of the resident without any documentary evidence having to be produced to the Authorised Dealer except for travel purposes outside the CMA where a passenger ticket needs to be 